# Numberphile Stable Marriage as LP

In this awesome [Numberphile video](https://www.youtube.com/watch?v=Qcv1IqHWAzg) the stable marriage problem is described, along with the Gale-Shapley algorithm for how to solve it. Here I take the lovely Jane Austen inspired problem and attempt to solve it as an LP in python using the PuLP package. 


Before we dive totally into the code, below I have used pandas dataframes to represent the preferences of our characters in the dating pool. We have 8 beloved Pride and Prejudice characters, 4 men and 4 women and there preferences are represented below. 

In [1]:
import numpy as np
import pandas as pd

womenPref = pd.DataFrame({
    "Charlotte" : ['Bingley' ,'Darcy', 'Collins', 'Wickham'],
    "Elizabeth" : ['Wickham', 'Darcy', 'Bingley', 'Collins'],
    "Jane" : ['Bingley', 'Wickham', 'Darcy', 'Collins'],
    "Lydia" : ['Bingley','Wickham','Darcy','Collins']})
menPref = pd.DataFrame({
    "Bingley" :['Jane','Elizabeth','Lydia','Charlotte'],
    "Collins" :['Jane','Elizabeth','Lydia','Charlotte'],
    "Darcy" :['Elizabeth','Jane','Charlotte','Lydia'],
    "Wickham" :['Lydia','Jane','Elizabeth','Charlotte']})

In [2]:
womenPref

,Charlotte,Elizabeth,Jane,Lydia
0,Bingley,Wickham,Bingley,Bingley
1,Darcy,Darcy,Wickham,Wickham
2,Collins,Bingley,Darcy,Darcy
3,Wickham,Collins,Collins,Collins


In [3]:
menPref

,Bingley,Collins,Darcy,Wickham
0,Jane,Jane,Elizabeth,Lydia
1,Elizabeth,Elizabeth,Jane,Jane
2,Lydia,Lydia,Charlotte,Elizabeth
3,Charlotte,Charlotte,Lydia,Charlotte


## Formulating our LP

### The decision variables 
We have 16 decision variables in total (4 men * 4 women or M men and W women) that make up all possible pairs. 

$$
  \begin{equation}
    x_{m w}=
    \begin{cases}
      1, & \text{if man m and woman w are matched}\\
      0, & \text{otherwise}
    \end{cases}
  \end{equation}
$$

### The constraints 
We have two types of constraints: (1.) matching constraints that ensure that each person is matched with one other person of the opposite sex and (2.) stability constraints which ensure that our "matches" can do no better because if man m' and women w' have not been paired then one of them has been paired with a partner that they prefer more.

#### Matching constraints 

Our first constraint specifies that each woman is matched to one man
$$
\begin{equation}
\sum_{m=1}^M x_{m w} = 1 \text{  for woman = w within [Charlotte, Elizabeth, Jane, Lydia]}
\end{equation}
$$

Our second constraint specifies that each man is matched to one woman
$$
\begin{equation}
\sum_{w=1}^W x_{m w} = 1 \text{  for man = m within [Bingley, Collins, Darcy, Wickham]}
\end{equation}
$$

#### Stability constraints 
We have 16 stability constraints (one for each potential pair). These basically say that for a given pair (m,w) if either m or w is married to someone who they prefer less to m or w the pair is unstable. So algebraically we have something like:


$$
\begin{equation}
x_{m w} + \sum_{w'= w - 1}^W x_{m w'} + \sum_{m'= m - 1}^M x_{m' w} \leq 1 \text{  for every pair (m,w)}
\end{equation}
$$


In [4]:
Women = ['Charlotte', 'Elizabeth', 'Jane', 'Lydia']
Men = ['Bingley', 'Collins', 'Darcy', 'Wickham']

#create a list of tuples containing all possible strokes and swimers
combos = [(m,w) for m in Men for w in Women]

print(combos)

[('Bingley', 'Charlotte'), ('Bingley', 'Elizabeth'), ('Bingley', 'Jane'), ('Bingley', 'Lydia'), ('Collins', 'Charlotte'), ('Collins', 'Elizabeth'), ('Collins', 'Jane'), ('Collins', 'Lydia'), ('Darcy', 'Charlotte'), ('Darcy', 'Elizabeth'), ('Darcy', 'Jane'), ('Darcy', 'Lydia'), ('Wickham', 'Charlotte'), ('Wickham', 'Elizabeth'), ('Wickham', 'Jane'), ('Wickham', 'Lydia')]


In [5]:
# Import PuLP modeler functions
from pulp import *

In [6]:
#Create the 'prob' variable to contain the problem data
prob = LpProblem("Stable_Marriage", LpMaximize)

In [7]:
#Create the Xmw variable
pair = LpVariable.dicts("pair", (Men, Women),0,1,LpInteger)

In [8]:
#Create a constraint ensuring that man is assigned to only one woman
for m in Men:
    prob += lpSum(pair[m][w] for w in Women) == 1, ""
    
#Create a constraint ensuring that  womman is assigned to only one man
for w in Women:
    prob += lpSum(pair[m][w] for m in Men) == 1, ""

In [10]:
#Now we formulate our stability
# Loop through every combo 
for (m,w) in combos:
    menVar = []
    womenVar = []
    
    #Add the initial pair to the respective lists
    menVar.append(m)
    womenVar.append(w)
    
    ##
    #For the Woman: look at her preferences and collect her potential pairs that are lowerer than the initial pair 
    ##
    sel_row = list(np.where(womenPref[w] == m)[0])[0] #Get the row index of initial m
    #Loop through woman w's men that are lower than m and add those men and w to the menVar and womenVar lists
    for i in range(sel_row+1,4): 
        lowerPrefMan = womenPref.loc[[i],[w]].to_numpy()[0][0]
        menVar.append(lowerPrefMan)
        womenVar.append(w)

    ##
    #For the Man: look at his preferences and collect his potential pairs that are lowerer than the initial pair 
    ##
    sel_row = list(np.where(menPref[m] == w)[0])[0] #Get the row index of initial w

    #Loop through man m's women that are lower than w and add those wommen and m to the menVar and womenVar lists
    for i in range(sel_row+1,4): 
        lowerPrefWoman = menPref.loc[[i],[m]].to_numpy()[0][0]
        menVar.append(m)
        womenVar.append(lowerPrefWoman)
    
    #We zip our pair by combining menVar and womenVar
    stabilityPairs = zip(menVar,womenVar)
    
    #Add the stability constraint
    prob += lpSum(pair[m][w] for (m, w) in stabilityPairs) <= 1, ""

In [11]:
# The problem data is written to an .lp file
prob.writeLP("Model.lp")

In [12]:
prob.solve()

1

In [13]:
#Print the variables
for v in prob.variables():
    print(v.name, "=", v.varValue)

__dummy = None
pair_Bingley_Charlotte = 0.0
pair_Bingley_Elizabeth = 0.0
pair_Bingley_Jane = 1.0
pair_Bingley_Lydia = 0.0
pair_Collins_Charlotte = 1.0
pair_Collins_Elizabeth = 0.0
pair_Collins_Jane = 0.0
pair_Collins_Lydia = 0.0
pair_Darcy_Charlotte = 0.0
pair_Darcy_Elizabeth = 1.0
pair_Darcy_Jane = 0.0
pair_Darcy_Lydia = 0.0
pair_Wickham_Charlotte = 0.0
pair_Wickham_Elizabeth = 0.0
pair_Wickham_Jane = 0.0
pair_Wickham_Lydia = 1.0
